In [8]:
import pickle as pkl
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import entropy
import sklearn.metrics as metrics
from sktr_update.core import compare_stochastic_vs_argmax_random_indices, train_sktr, test_sktr_parallel, test_sktr
import pickle as pkl
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sktr_update.utils import prepare_df_from_dataset
from denoisers.ConditionalUnetDenoiser import ConditionalUnetDenoiser
from denoisers.ConditionalUnetMatrixDenoiser import ConditionalUnetMatrixDenoiser
from utils.graph_utils import get_process_model_reachability_graph_transition_matrix, \
    get_process_model_petri_net_transition_matrix, get_process_model_reachability_graph_transition_multimatrix
from utils.pm_utils import discover_dk_process, remove_duplicates_dataset, pad_to_multiple_of_n
from utils.Config import Config
from dataset.dataset import SaladsDataset
from ddpm.ddpm_multinomial import Diffusion
import os
import json
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from utils.pm_utils import conformance_measure
from scipy.stats import wasserstein_distance
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

ImportError: cannot import name 'test_sktr' from 'sktr_update.core' (D:\Projects\trace-denoise\src\sktr_update\core.py)

In [2]:
def load_experiment_config(target_dir):
    config_path = os.path.join(target_dir, "cfg.json")
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            return Config(**json.load(f))
    else:
        st.warning("Configuration file not found.")
        return None


def load_experiment_data_and_model(target_dir, cfg):
    with open(cfg.data_path, "rb") as f:
        base_dataset = pkl.load(f)
    dataset = SaladsDataset(base_dataset['target'], base_dataset['stochastic'])
    train_dataset, test_dataset = train_test_split(dataset, train_size=cfg.train_percent, shuffle=True,
                                                   random_state=cfg.seed)
    dk_process_model, dk_init_marking, dk_final_marking = discover_dk_process(train_dataset, cfg,
                                                                              preprocess=remove_duplicates_dataset)
    diffuser = Diffusion(noise_steps=cfg.num_timesteps, device=cfg.device)
    if cfg.enable_matrix:
        rg_nx, rg_transition_matrix = get_process_model_petri_net_transition_matrix(dk_process_model,
                                                                                    dk_init_marking, dk_final_marking)
        rg_transition_matrix = torch.tensor(rg_transition_matrix, device=cfg.device).unsqueeze(0).float()
        rg_transition_matrix = pad_to_multiple_of_n(rg_transition_matrix)
        denoiser = ConditionalUnetMatrixDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes,
                                                 max_input_dim=dataset.sequence_length,
                                                 transition_dim=rg_transition_matrix.shape[-1],
                                                 matrix_out_channels=rg_transition_matrix.shape[0],
                                                 device=cfg.device).to(cfg.device).float()
    else:
        rg_transition_matrix = torch.randn((cfg.num_classes, 2, 2)).to(cfg.device)
        denoiser = ConditionalUnetDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes,
                                           max_input_dim=dataset.sequence_length,
                                           device=cfg.device).to(cfg.device).float()
    ckpt_path = os.path.join(target_dir, "best.ckpt")
    denoiser.load_state_dict(torch.load(ckpt_path, map_location=cfg.device)['model_state'])
    final_res_path = os.path.join(target_dir, "final_results.json")
    final_res = None
    if os.path.exists(final_res_path):
        with open(final_res_path, "r") as f:
            final_res = json.load(f)
    else:
        pass

    return (train_dataset, test_dataset, dk_process_model, dk_init_marking, dk_final_marking, rg_transition_matrix,
            diffuser, denoiser, final_res)

In [3]:
def evaluate_dataset(denoiser, diffuser, cfg, loader):
    results_accumulator = {'x': [], 'y': [], 'x_hat': []}
    l = len(loader)
    with torch.no_grad():
        for i, (x, y) in tqdm(enumerate(loader)):
            x = x.permute(0, 2, 1).to(cfg.device).float()
            y = y.permute(0, 2, 1).to(cfg.device).float()
            x_hat, matrix_hat, loss, seq_loss, mat_loss = \
                diffuser.sample_with_matrix(denoiser, y.shape[0], cfg.num_classes, denoiser.max_input_dim,
                                            rg_transition_matrix.shape[-1], rg_transition_matrix, x, y,
                                            cfg.predict_on)
            results_accumulator['x'].append(x)
            results_accumulator['y'].append(y)
            results_accumulator['x_hat'].append(x_hat.permute(0, 2, 1))

            x_argmax = torch.argmax(torch.cat(results_accumulator['x'], dim=0), dim=1).to('cpu')
            y_cat = torch.cat(results_accumulator['y'], dim=0)
            x_hat_logit = torch.cat(results_accumulator['x_hat'], dim=0)

            x_argmax_flat = x_argmax.reshape(-1).to('cpu')
            x_hat_flat = x_hat_logit.reshape(-1, cfg.num_classes).to('cpu')
            x_hat_prob_flat = torch.softmax(x_hat_flat, dim=1).to('cpu')
            x_hat_argmax_flat = torch.argmax(x_hat_prob_flat, dim=1).to('cpu')
            x_hat_prob = torch.softmax(x_hat_logit, dim=2).to('cpu')
            x_hat_argmax = torch.argmax(x_hat_prob, dim=2)

            w2 = np.mean([wasserstein_distance(xi, xhi) for xi, xhi in zip(x_argmax, x_hat_argmax)])
            accuracy = accuracy_score(x_argmax_flat, x_hat_argmax_flat)
            precision = precision_score(x_argmax_flat, x_hat_argmax_flat, average='macro', zero_division=0)
            recall = recall_score(x_argmax_flat, x_hat_argmax_flat, average='macro', zero_division=0)
            f1 = f1_score(x_argmax_flat, x_hat_argmax_flat, average='macro', zero_division=0)
            alignments = 0
            # alignments = np.mean(
            #     conformance_measure(x_hat_argmax, process_model, init_mark, final_mark, cfg.activity_names,
            #                         limit=1000, remove_duplicates=True, approximate=False)
            # )

    return results_accumulator, (accuracy, precision, recall, f1, w2, alignments)


def evaluate_dataset_basic(denoiser, diffuser, cfg, loader):
    denoiser.eval()
    results = []
    with torch.no_grad():
        for i, (x, y) in tqdm(enumerate(loader)):
            x = x.permute(0, 2, 1).to(cfg.device).float()
            y = y.permute(0, 2, 1).to(cfg.device).float()
            x_hat, matrix_hat, loss, seq_loss, mat_loss = \
                diffuser.sample_with_matrix(denoiser, y.shape[0], cfg.num_classes, denoiser.max_input_dim,
                                            rg_transition_matrix.shape[-1], rg_transition_matrix, x, y,
                                            cfg.predict_on)
            results.append(x_hat)
    return results

In [ ]:
datasets = ["50_salads_unified.pkl", "gtea_unified.pkl", "breakfast_unified.pkl", "bpi12_reduced.pkl", "bpi19_reduced.pkl"]
for dataset in datasets:
    with open(rf"../data/pickles/{dataset}", "rb") as f:
        data = pkl.load(f)
    target, source = data['target'], data['stochastic']
    target_for_sktr = [np.argmax(t, axis=1) for t in target]
    source_for_sktr = [np.array(t).T for t in source]
    target_train, target_test = train_test_split(target_for_sktr, test_size=0.75, shuffle=True, random_state=42)
    source_train, source_test = train_test_split(source_for_sktr, test_size=0.75, shuffle=True, random_state=42)
    df_train, softmax_train = prepare_df_from_dataset(target_train, source_train)
    df_test, softmax_test = prepare_df_from_dataset(target_test, source_test)
    model, prob_dict = train_sktr(df_train, softmax_train, 60, True, 42, 1.0, True, [0.1, 0.3, 0.6])
    recovery_results_df, alignment_results_df, model = test_sktr(df_test, softmax_test, model, 'logarithmic', (0.1, 0.3, 0.6), 0.6, True, prob_dict, False, 1.0, 4, 0.01, 60, True, 42)
    case_lists = recovery_results_df.groupby("case:concept:name")[["sktr_pred", "argmax_pred", "ground_truth"]].apply(lambda g: g.values.tolist()).tolist()
    sktr_result = [[int(x[0]) for x in y if x[0] is not None] for y in case_lists]
    sktr_gt = [[int(x[2]) for x in y if x[0] is not None] for y in case_lists]
    with open(f"{dataset}_sktr_accumulator.pkl", "wb") as f:
        pkl.dump({'sktr': sktr_result, 'gt': sktr_gt}, f)
    accs = [metrics.accuracy_score(t, s) for t, s in zip(sktr_gt, sktr_result)]
    prec = [metrics.precision_score(t, s, average='macro', zero_division=0) for t, s in zip(sktr_gt, sktr_result)]
    recs = [metrics.recall_score(t, s, average='macro', zero_division=0) for t, s in zip(sktr_gt, sktr_result)]
    with open(f"{dataset}_sktr_accumulator.pkl", "wb") as f:
        pkl.dump([accs, prec, recs], f)

In [ ]:
target_dirs = ["gtea", "breakfast", "bpi12", "bpi19"]
pad_tokens = {"gtea": 11, "breakfast": 48, "bpi12": 24, "bpi19": 42}

In [ ]:
for dir in target_dirs:
    target_dir = rf"D:\Projects\trace-denoise\final_runs\{dir}"
    cfg = load_experiment_config(target_dir)
    cfg.device = "cuda:0"
    train_dataset, test_dataset, dk_process_model, dk_init_marking, dk_final_marking, rg_transition_matrix, diffuser, denoiser, final_res = load_experiment_data_and_model(
        target_dir, cfg)
    test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)
    accumulator, measures = evaluate_dataset(denoiser, diffuser, cfg, test_loader)
    with open(f"{dir}_ddtr_accumulator.pkl", "wb") as f:
        pkl.dump(accumulator, f)
    accs, pre, rec = [], [], []
    pad_token = pad_tokens[dir]
    for x, y, x_hat in zip(accumulator['x'], accumulator['y'], accumulator['x_hat']):
        for tdk, tsk, that in zip(x, y, x_hat):
            dk = torch.argmax(tdk, dim=0).cpu().numpy()
            hat = torch.argmax(torch.softmax(that, dim=1), dim=1).cpu().numpy()
            accs.append(metrics.accuracy_score(dk[dk != pad_token], hat[dk != pad_token]))
            pre.append(metrics.precision_score(dk[dk != pad_token], hat[dk != pad_token], average='macro', zero_division=0))
            rec.append(metrics.recall_score(dk[dk != pad_token], hat[dk != pad_token], average='macro', zero_division=0))
    with open(f"{dir}_ddtr_measures.pkl", "wb") as f:
        pkl.dump([accs, pre, rec], f)